# Snake Game

In [1]:
from random import randint
from math import sqrt

x = int(input('Digite a coordenada x: '))
y = int(input('Digite a coordenada y: '))
posicao_objetivo = {}
qtd_objetivo = int(input('Digite o numero de objetivos: '))
mapa = [ [' ' for j in range(0, x)]  for i in range(0, y)]

for i in range(0,qtd_objetivo):
    posicao_objetivo_x = randint(0,x-1)
    posicao_objetivo_y = randint(0,y-1)
    posicao_objetivo[i]=[posicao_objetivo_x,posicao_objetivo_y]
posicao_objetivo

Digite a coordenada x: 10
Digite a coordenada y: 10
Digite o numero de objetivos: 5


{0: [3, 4], 1: [5, 7], 2: [8, 4], 3: [6, 9], 4: [0, 3]}

### Estado Inicial
Cria o estado inicial do agente.
Preenche a posição dos objetivos.

In [2]:
estado = {'mapa': mapa, 'x': (x/2)-1, 'y': (y/2)-1, 'passo':0, 'posicao_objetivo':posicao_objetivo}
for i in posicao_objetivo:
    mapa[posicao_objetivo[i][0]][posicao_objetivo[i][1]] = "#"
mapa[estado['x']][estado['y']] = 'X'
estado

{'mapa': [[' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', 'X', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' '],
  [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#'],
  [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']],
 'passo': 0,
 'posicao_objetivo': {0: [3, 4], 1: [5, 7], 2: [8, 4], 3: [6, 9], 4: [0, 3]},
 'x': 4,
 'y': 4}

### Função Sucessores
Retorna uma lista com todos os estados possíveis do agente, dado o atual estado.

In [3]:
import copy

def sucessores(estado, x, y):
    lista_sucessores = []
    
    # Movimenta o agente para cima ou para baixo
    for i in [-1, 1]:
        novo_x = estado['x']+i
        novo_y = estado['y']
        
        if novo_x >= 0 and novo_x < x:
            sucessor = copy.deepcopy(estado) 
            sucessor['mapa'] = copy.deepcopy(estado['mapa'])

            sucessor['mapa'][novo_x][novo_y] = '*'
            sucessor['x'] = novo_x
            sucessor['y'] = novo_y
            sucessor['passo'] += 1

            lista_sucessores.append(sucessor)
    
    # Movimenta o agente para esquerda ou direita
    for j in [-1, 1]:
        novo_x = estado['x']
        novo_y = estado['y']+j

        if novo_y >= 0 and novo_y < y:
            sucessor = copy.deepcopy(estado) 
            sucessor['mapa'] = copy.deepcopy(estado['mapa'])

            sucessor['mapa'][novo_x][novo_y] = '*'
            sucessor['x'] = novo_x
            sucessor['y'] = novo_y
            sucessor['passo'] += 1

            lista_sucessores.append(sucessor)
            
    return lista_sucessores

### Heurística
Com base no atual estado, determinar o menor caminho para um dos objetivos. Após isso, escolher o caminho mais próximo do objetivo escolhido. O menor caminho para o objetivo será o escolhido.

In [4]:
def menor_caminho (estado, sucessores_lista):

    possivel_estado = []
    lista_menor_valor = []
    for xy in sucessores_lista:
        possivel_estado.append([xy['x'], xy['y']])

    #print(possivel_estado)

    for pos_obj in posicao_objetivo.keys():
        distAB = sqrt((estado['x']-posicao_objetivo[pos_obj][0])**2) + ((estado['y']-posicao_objetivo[pos_obj][1])**2)
        lista_menor_valor.append([distAB, pos_obj])

    #print(lista_menor_valor)

    for i in range(0,len(lista_menor_valor)):
        if i == 0:
            aux = lista_menor_valor[i][0]
            chave_menor_valor = lista_menor_valor[i][1]
        if lista_menor_valor[i][0] < aux:
            chave_menor_valor = lista_menor_valor[i][1]

    #print(chave_menor_valor)

    # zera a lista de menor valor
    lista_menor_valor = []

    # calcula quais dos estados deixará o agente mais proximo do objetivo
    for pe in possivel_estado:
        distAB = sqrt((pe[0]-posicao_objetivo[chave_menor_valor][0])**2) + ((pe[1]-posicao_objetivo[chave_menor_valor][1])**2)
        lista_menor_valor.append(distAB)

    # verifica o valor da menor distância
    for i in range(0,len(lista_menor_valor)):
        if i == 0:
            obj_proximo = lista_menor_valor[i]
        else:
            if lista_menor_valor[i] < obj_proximo:
                obj_proximo = lista_menor_valor[i]

    # verifica posição da menor distância
    for key, value in enumerate(lista_menor_valor):
        if value == obj_proximo:
            posic_prox_estado = key

    #print(posic_prox_estado)

    return posic_prox_estado

### Busca Gulosa

In [5]:
def busca_gulosa(estado, mapa):
    obj_atingido = 'x'
    pegadas = []
    end = 0
    while True:
        
        # calcula os sucessores do estado atual
        sucessores_lista = sucessores(estado, x, y)
        
        # calcula o menor caminho para o objetivo dado o estado atual
        key_menor_caminho = menor_caminho(estado, sucessores_lista)
        
        estado['x'] = sucessores_lista[key_menor_caminho]['x']
        estado['y'] = sucessores_lista[key_menor_caminho]['y']
        estado['passo'] += 1
        pegadas.append([estado['x'], estado['y']])
        
        # percorre a lista de objetivos
        for d in posicao_objetivo.keys():
            # se eu já passei em um objetivo, vou guardar na variável obj_atingido
            if posicao_objetivo[d] in pegadas:
                obj_atingido = d
                end += 1
        
        # se eu atingi um objetivo, vou retirar da lista e colocar o valor na variavel.
        if obj_atingido != 'x':
            del posicao_objetivo[obj_atingido]
            obj_atingido = 'x'
        
        if mapa[estado['x']][estado['y']] != '#':
            mapa[estado['x']][estado['y']] = '*'
        
        print(mapa)
        print(estado['passo'])
        
        if end == qtd_objetivo:
        #end +=1
            return True
            
busca_gulosa(estado, mapa)

[[' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', 'X', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#'], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]
1
[[' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', '*', '#', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', 'X', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#'], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '], [' ', ' 

True